<a href="https://colab.research.google.com/github/hoanglam372000/Applied-Data-Science-Group-7/blob/nguyenthiman/TXL_2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Import

In [ ]:
!pip install contractions
!pip install emoji
!pip install autocorrect

In [ ]:
import pandas as pd
import nltk
import string
import re
import contractions
from nltk.corpus import wordnet
from nltk.stem import PorterStemmer, WordNetLemmatizer
nltk.download('punkt')
nltk.download('wordnet')
nltk.download('averaged_perceptron_tagger')
nltk.download('names')
import emoji
from bs4 import BeautifulSoup
from autocorrect import Speller
names = nltk.corpus.names.words('male.txt')+nltk.corpus.names.words('female.txt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!
[nltk_data] Downloading package names to /root/nltk_data...
[nltk_data]   Package names is already up-to-date!


# TXTXL

còn nhiềnhiều ý:
từ viết tắt

emoij

emoij bị dính  hi:))) :)<3

kí tự đặc biệt (vì tránh emoij)


In [ ]:
tweet = " #gamegame36 didn't My mom just texted me and told me that Rodney was *** chasing?? and fireflies... in (their) :)) backyard. Awwwww I'm miss him!!! @hoanglam 123 ://    http://twitpic.com/66pn1"
tweet2 = 'theira :)) <3'
def preprocessing_word(word):
  word = contractions.fix(word)
  print(1,word)
  temp=[ "<html_tag>",'<URL>','<HIDDEN>','<tag>',"<hash_tag>"]
  if word in temp:
    return word
  #số theo từ
  if word.isdigit():  
    return '<NUMBER>'
  try: 
    float(word)
    return '<NUMBER>'
  except:
    pass
  if word in names:
    return '<Name>'
  # emoij
  # emoij dính  hi:))) :)<3

  word=word.lower() # viết thườngthường
  spell_corrector = Speller(lang='en') #sửa chính tả theo từng từ 
  word=spell_corrector(word)

  # correct_words=[]
  # spell_corrector = SpellChecker()
  # misSpelled_words = spell_corrector.unknown(word.split())
  # for each_word in word.split():
  #   if each_word in misSpelled_words:
  #     right_word = spell_corrector.correction(each_word)
  #     correct_words.append(right_word)
  #   else:
  #     correct_words.append(each_word)
  # word = ' '.join(correct_words)

  print(2,word)
  # print('correct',word)
  # rút gọn theo từ
  lemmatizer = WordNetLemmatizer()
  wordnet_map = {"N":wordnet.NOUN, "V":wordnet.VERB, "J":wordnet.ADJ, "R":wordnet.ADV}
  pos_tagged = nltk.pos_tag(word.split())
  
  word = [lemmatizer.lemmatize(word, wordnet_map.get(pos[0], wordnet.NOUN)) for word, pos in pos_tagged]
  word = ' '.join(word)
  print(3,word)
  word=word.translate(str.maketrans('', '', string.punctuation)) # chấm câu theo từ (vì tránh emoij)
  if word==''or word ==' ':
    return "<Special_Character>"
  if 'not' not in word and len(word.split())!=1:
    return word.split()[0]+'_'
  return word.replace(' ','_')

def preprocessing_text(tweet):
   raw_text = str(tweet).strip().replace('\n', ' ')#.lower()
   new_tweet = raw_text.split()
   len_=len(new_tweet)
   index2word = { w:new_tweet[w] for w in range(len_)}
   print(index2word)
   parser = BeautifulSoup(raw_text, "html.parser")
   raw_text = parser.get_text(separator = "<html_tag>")
   raw_text = re.sub(pattern=r'https?://\S+|www\.\S+', repl='<URL>', string=raw_text)
   raw_text = re.sub(pattern='\*+', repl='<HIDDEN>', string=raw_text)
   raw_text = re.sub(pattern=r'@\w+', repl='<tag>', string=raw_text)
   raw_text = re.sub(pattern=r'#\w+', repl='<hash_tag>', string=raw_text)
   word_list=raw_text.split()
   
   return dict(zip([*range(len_)],map(preprocessing_word,word_list)))
  #  return list(map(preprocessing_word,word_list))
   # (" ".join([word_list]))

# preprocessing_text(tweet)
preprocessing_word("texted")

1 texted
2 tested
3 test


'test'